In [1]:
from src.rel_prop.min_max_rel_model import MinMaxModel, Nested_Regressor
from src.models.Binary_Mnist_Model import Montavon_Classifier
import numpy as np
from src.rel_prop.minmax_utils import get_higher_relevances
import matplotlib.pyplot as plt
import time

In [2]:
mc = Montavon_Classifier(class_nb=8, load_model=True)
mc.set_data(test_size=0.2)
mc.set_model()
mc.model.summary()
time0=time.time()
epochs = 300
#epochs = 600
mc.fit_model(epochs=300, batch_size=32)
time1 = time.time()-time0
print("Duration to fit Montavon Classifier: {}".format(time1))

Train images: 21840, train labels: 21840
Load model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 400)               313600    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               40000     
_________________________________________________________________
dense_2 (Dense)              (None, 400)               40000     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 400       
Total params: 394,000
Trainable params: 353,600
Non-trainable params: 40,400
_________________________________________________________________
Model has been load, no need to train!
Duration to fit Mont

In [3]:
mc.evaluate(batch_size=32)

171/171 [==============================] - 0s 1ms/step - loss: 0.5882 - acc: 0.9555


0.9554945230484009

In [4]:
#mc.non_trivial_accuracy()

In [5]:
# use to recalc relevances for the training of the min max model. Therefore set recalc_rel to True


In [6]:
minmax = MinMaxModel(classifier=mc, use_higher_rel=True)



Started to collect relevances to train min-max-model!
Load relevances to train min-max-model from local directory!
Created MinMaxModel


In [7]:
nr_train_images = minmax.nr_train_images
true_relevances = minmax.true_relevances
higher_relevances = minmax.higher_relevances

time2 = time.time()
minmax.train_min_max(pretrained=False)
time3 = time.time()-time2
print("Duration to train Min-Max-Model: {}".format(3))

Start Training of Min-Max-Model
Created nested regressor for neuron with index 0
Created nested regressor for neuron with index 1
Created nested regressor for neuron with index 2
Created nested regressor for neuron with index 3
Created nested regressor for neuron with index 4
Created nested regressor for neuron with index 5
Created nested regressor for neuron with index 6
Created nested regressor for neuron with index 7
Created nested regressor for neuron with index 8
Created nested regressor for neuron with index 9
Created nested regressor for neuron with index 10
Created nested regressor for neuron with index 11
Created nested regressor for neuron with index 12
Created nested regressor for neuron with index 13
Created nested regressor for neuron with index 14
Created nested regressor for neuron with index 15
Created nested regressor for neuron with index 16
Created nested regressor for neuron with index 17
Created nested regressor for neuron with index 18
Created nested regressor for

In [8]:
from src.plotting.plot_funcs import plotly_mnist_image
import random 

while True:
    idx = random.randint(0, minmax.classifier.test_images.shape[0])
    if mc.predict_test_image(idx) == 1 and mc.test_labels[idx] ==1:
        break
plotly_mnist_image(minmax.classifier.test_images[idx])
final_relevance, zplus_relevance = minmax.min_max_rel_prop(idx)

NameError: name 'minmax' is not defined

In [ ]:
final_relevance_ = final_relevance
plotly_mnist_image(final_relevance_)


In [ ]:
import numpy as np 
# Run Min-Max Model without higher relevances 
minmax_nobias = MinMaxModel(classifier=mc, use_higher_rel=False)

In [ ]:
minmax_nobias.train_min_max(pretrained=True)

In [ ]:
while True:
    idx = random.randint(0, minmax_nobias.classifier.test_images.shape[0])
    if mc.predict_test_image(idx) == 1 and mc.test_labels[idx] ==1:
        break
plotly_mnist_image(minmax_nobias.classifier.test_images[idx])
final_relevance, zplus_relevance = minmax_nobias.min_max_rel_prop(idx)

In [ ]:
final_relevance_ = final_relevance
plotly_mnist_image(final_relevance_)

In [ ]:
final_relevance_ = final_relevance
plotly_mnist_image(final_relevance_)

In [ ]:
import numpy as np 
# Run Min-Max Model without higher relevances 
minmax_nobias = MinMaxModel(classifier=mc, use_higher_rel=False)

In [ ]:
minmax_nobias.train_min_max(pretrained=True)

In [ ]:
while True:
    idx = random.randint(0, minmax_nobias.classifier.test_images.shape[0])
    if mc.predict_test_image(idx) == 1 and mc.test_labels[idx] ==1:
        break
plotly_mnist_image(minmax_nobias.classifier.test_images[idx])
final_relevance, zplus_relevance = minmax_nobias.min_max_rel_prop(idx)

In [ ]:
final_relevance_ = final_relevance
plotly_mnist_image(final_relevance_)

In [9]:
final_relevance_ = final_relevance
plotly_mnist_image(final_relevance_)

NameError: name 'final_relevance' is not defined

In [10]:
import numpy as np 
# Run Min-Max Model without higher relevances 
minmax_nobias = MinMaxModel(classifier=mc, use_higher_rel=False)

Started to collect relevances to train min-max-model!
Load relevances to train min-max-model from local directory!


TypeError: cannot unpack non-iterable NoneType object

In [11]:
minmax_nobias.train_min_max(pretrained=True)

NameError: name 'minmax_nobias' is not defined

In [12]:
while True:
    idx = random.randint(0, minmax_nobias.classifier.test_images.shape[0])
    if mc.predict_test_image(idx) == 1 and mc.test_labels[idx] ==1:
        break
plotly_mnist_image(minmax_nobias.classifier.test_images[idx])
final_relevance, zplus_relevance = minmax_nobias.min_max_rel_prop(idx)

NameError: name 'minmax_nobias' is not defined

In [13]:
final_relevance_ = final_relevance
plotly_mnist_image(final_relevance_)

NameError: name 'final_relevance' is not defined